In [1]:
import pandas as pd
import numpy as np

In [2]:
# IMPORTAÇÃO DA BASE DE EMPRESAS ROTULADAS
empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\empresas_rotuladas.csv', dtype=str, usecols=['cnpj'])
empresas_rotuladas_df

,cnpj
0,00000942000170
1,00001102000121
2,00005201000181
3,00005201000262
4,00010501000159
...,...
1559938,98593668000194
1559939,98669997000252
1559940,98670235000195
1559941,98670656000116


### Junção com a base de pessoas que receberam recursos do governo federal:

In [3]:
# IMPORTAÇÃO DA BASE DE RECEBIMENTO DE RECURSOS DO GOVERNO
recursos_governo_df = pd.read_csv(r'C:\DATASETS\Recebimento Recursos Governo Federal\recebimento_recursos_governo-consolidado.csv', dtype=str, usecols=['Código Favorecido', 'recebimento_recurso_governo_federal'])
recursos_governo_df

,Código Favorecido,recebimento_recurso_governo_federal
0,***000000**,-468.52
1,***000000**,4582.54
2,***000000**,218793.06
3,***000000**,800.0
4,***000000**,6400.0
...,...,...
3805117,98671597000362,208.1
3805118,98674401000121,4000.0
3805119,98675598000113,11160.0
3805120,98714876000102,349600.0


In [4]:
# Só nos interessa os CNPJs:
recursos_governo_df = recursos_governo_df[recursos_governo_df['Código Favorecido'].str.len()==14]
recursos_governo_df.shape

(355625, 2)

In [5]:
# Rotula as empresas
empresas_rotuladas_df['empresa_recurso_recebido_governo_federal'] = empresas_rotuladas_df.merge(recursos_governo_df, left_on='cnpj', right_on='Código Favorecido', how='left')['recebimento_recurso_governo_federal'].astype(float)
empresas_rotuladas_df = empresas_rotuladas_df.fillna({'empresa_recurso_recebido_governo_federal': 0})
print(empresas_rotuladas_df[empresas_rotuladas_df.empresa_recurso_recebido_governo_federal>0].shape)
print(empresas_rotuladas_df.empresa_recurso_recebido_governo_federal.sum())
print(empresas_rotuladas_df)

(15370, 2)
1772834232819.9192
                   cnpj  empresa_recurso_recebido_governo_federal
0        00000942000170                                      0.00
1        00001102000121                                      0.00
2        00005201000181                                      0.00
3        00005201000262                                      0.00
4        00010501000159                                      0.00
...                 ...                                       ...
1559938  98593668000194                                   5818.65
1559939  98669997000252                                      0.00
1559940  98670235000195                                      0.00
1559941  98670656000116                                      0.00
1559942  98674195000150                                      0.00

[1559943 rows x 2 columns]


In [6]:
# Apenas para comparar com a base completa de recursos recebidos por PJ:
recursos_governo_df['recebimento_recurso_governo_federal'].astype(float).sum()

2742594509955.969

In [7]:
1772834232819.9192 * 100 / 2742594509955.969

64.64077086074167

As empresas rotuladas correspondem a 64% dos recursos recebidos por empresas.

In [8]:
# Salvando resultado:
empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\empresas_rotuladas-recebimento_recurso_governo_federal.csv', index = False)